<a href="https://colab.research.google.com/github/don-rukshan/Stock-Price-Prediction/blob/master/Machine_Learning_Stock_Market_Prediction_for_n_days.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install quandl

In [0]:
# This program predicts stock prices by using machine learning models
# install the dependencies

import quandl
import numpy as np
import pandas_datareader as web
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [0]:
# Get the stock data
df = quandl.get("WIKI/AMZN")
#df = web.DataReader('AAPL', data_source= 'yahoo', start='2012-01-01', end = '2019-12-17')
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
1997-05-16,22.38,23.75,20.50,20.75,1225000.0,0.0,1.0,1.865000,1.979167,1.708333,1.729167,14700000.0
1997-05-19,20.50,21.25,19.50,20.50,508900.0,0.0,1.0,1.708333,1.770833,1.625000,1.708333,6106800.0
1997-05-20,20.75,21.00,19.63,19.63,455600.0,0.0,1.0,1.729167,1.750000,1.635833,1.635833,5467200.0
1997-05-21,19.25,19.75,16.50,17.13,1571100.0,0.0,1.0,1.604167,1.645833,1.375000,1.427500,18853200.0
1997-05-22,17.25,17.38,15.75,16.75,981400.0,0.0,1.0,1.437500,1.448333,1.312500,1.395833,11776800.0


In [0]:
# get the adjusted close price
df = df[['Adj. Close']]
df.head()

,Adj. Close
Date,
1997-05-16,1.729167
1997-05-19,1.708333
1997-05-20,1.635833
1997-05-21,1.427500
1997-05-22,1.395833


In [0]:
# variable for predicting 'n' days in future
forecast_out = 30
# create another column (the target or dependent variable) shifted 'n 'units up
df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)
df.tail()


,Adj. Close,Prediction
Date,,
2018-03-21,1581.86,NaN
2018-03-22,1544.10,NaN
2018-03-23,1495.56,NaN
2018-03-26,1555.86,NaN
2018-03-27,1497.05,NaN


In [0]:
### create the independent data set (x)
# convert the dataset to a numpy array
X = np.array(df.drop(['Prediction'],1))

#remove the last n rows
X = X[:-forecast_out]
print(X)

[[   1.72916667]
 [   1.70833333]
 [   1.63583333]
 ...
 [1350.47      ]
 [1338.99      ]
 [1386.23      ]]


In [0]:
### Create the dependent data set (y)
# Convert the df to a numpy array (All of the values including the NaN's)
y = np.array(df['Prediction'])
# get all of the y values except the last n rows
y = y[:-forecast_out]
print(y)


[1.54166667e+00 1.51583333e+00 1.58833333e+00 ... 1.49556000e+03
 1.55586000e+03 1.49705000e+03]


In [0]:
# split the data into 80% training and 20% testing
x_train, x_test, y_train , y_test = train_test_split(X,y,test_size=0.2)

In [0]:
# create and train the support vector machine regressor

svr_rbf = SVR(kernel='rbf', C=1e3, gamma = 0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [0]:
# Test te model: Score returns the coefficient of determination R^2 of the prediction.
# The best possible score is 1.0

svm_confidence = svr_rbf.score(x_test,y_test)
print("svm confidence: ", svm_confidence)

svm confidence:  0.9630983945667755


In [0]:
# create and train the linear regression model
lr = LinearRegression()
# train the model
lr.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
# Test te model: Score returns the coefficient of determination R^2 of the prediction.
# The best possible score is 1.0
lr_confidence = lr.score(x_test,y_test)
print("lr confidence: ", lr_confidence)

lr confidence:  0.9893180352178682


In [0]:
# Set x_forecast equal to the last 30 rows of the original data set from the Adj. Close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)


[[1414.51]
 [1451.05]
 [1461.76]
 [1448.69]
 [1468.35]
 [1482.92]
 [1484.76]
 [1500.  ]
 [1521.95]
 [1511.98]
 [1512.45]
 [1493.45]
 [1500.25]
 [1523.61]
 [1537.64]
 [1545.  ]
 [1551.86]
 [1578.89]
 [1598.39]
 [1588.18]
 [1591.  ]
 [1582.32]
 [1571.68]
 [1544.93]
 [1586.51]
 [1581.86]
 [1544.1 ]
 [1495.56]
 [1555.86]
 [1497.05]]


In [0]:
# print linear regression predictions for the next n days
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

# print support vector regressor predictions for the next n days
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[1502.61650975 1541.52035092 1552.92320092 1539.0076781  1559.93952039
 1575.45208104 1577.41111409 1593.63701829 1617.00700498 1606.39202697
 1606.89243216 1586.6632865  1593.90319126 1618.7743935  1633.71202053
 1641.54815274 1648.85193902 1677.63056044 1698.39205204 1687.52154798
 1690.52397907 1681.28245358 1669.95413201 1641.47362431 1685.74351255
 1680.79269532 1640.58993005 1588.90978636 1653.11070652 1590.49617725]
[1060.03055206 1551.06132865  682.93271464 1084.33312852  682.90000104
  682.90000104  682.90000104  682.90000104  682.90000104  682.90000104
  682.90000104  682.90000104  682.90000104  682.90000104  682.90000104
  682.90000104  682.90000104  682.90000104  682.90000104  682.90000104
  682.90000104  682.90000104  682.90000104  682.90000104  682.90000104
  682.90000104  682.90000104  682.90000104  682.90000104  682.90000104]
